# How To: Adding Hunting Bookmarks from Notebooks

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.8 - AzureML<br>
__Platforms Supported:__<br>
    -  Azure ML
__Data Source Required:__<br>
    -  no
    
### Description
The sample notebook shows how to add hunting bookmarks to Microsoft Sentinel through Jupyter notebooks.

In [ ]:
# Parameters for notebooks testing, can be ignored safely
test_run = False

In [ ]:
# Loading Python libraries
from azure.common.credentials import get_azure_cli_credentials
import requests
import json
import uuid
import pandas

In [ ]:
# Functions will be used in this notebook
def read_config_values(file_path):
    "This loads pre-generated parameters for Microsoft Sentinel Workspace"
    with open(file_path) as json_file:
        if json_file:
            json_config = json.load(json_file)
            return (json_config["tenant_id"],
                    json_config["subscription_id"],
                    json_config["resource_group"],
                    json_config["workspace_id"],
                    json_config["workspace_name"])
    return None

# Calling Microsoft Sentinel API, the same template can be used for calling other Azure REST APIs with different parameters.
# For different environments, such as national clouds, you may need to use different root_url, please contact with your admins.
# It can be ---.azure.us, ---.azure.microsoft.scloud, ---.azure.eaglex.ic.gov, etc.
def call_azure_rest_api(token, resource_name, request_body, bookmark_id, api_version):
    "Calling Microsoft Sentinel REST API"
    headers = {"Authorization": token, "content-type":"application/json" }
    provider_name = "Microsoft.OperationalInsights"
    provider2_name = "Microsoft.SecurityInsights"
    target_resource_name = resource_name
    api_version = api_version
    root_url = "https://management.azure.com"
    arm_rest_url_template = "{0}/subscriptions/{1}/resourceGroups/{2}/providers/{3}/workspaces/{4}/providers/{5}/{6}/{7}?api-version={8}"
    arm_rest_url = arm_rest_url_template.format(root_url, subscription_id, resource_group, provider_name, workspace_name, provider2_name, target_resource_name, bookmark_id, api_version)
    print(arm_rest_url)
    response = requests.put(arm_rest_url, headers=headers, data=request_body)
    return response

def display_result_name(response):
    "Default to display column - name, you may change it to other columns"
    column_name = "name"
    if response != None:
        entries = [item[column_name] for item in response.json()["value"]] 
        display(entries)

def display_result(response):
    "Display the result set as pandas.DataFrame"
    if response != None:
        df = pandas.DataFrame(response.json()["value"])
        display(df)

In [ ]:
# Calling the above function to populate Microsoft Sentinel workspace parameters
# The file, config.json, was generated by the system, however, you may modify the values, or manually set the variables
tenant_id, subscription_id, resource_group, workspace_id, workspace_name = read_config_values('config.json');

In [ ]:
# Azure CLI is used to get device code to login into Azure, you need to copy the code and open the DeviceLogin site.
# You may add [--tenant $tenant_id] to the command
if test_run == False:
    !az login --tenant $tenant_id --use-device-code

In [ ]:
# Extract access token, which will be used to access Microsoft Sentinel Watchlist API for your Watchlist data.  
credentials, sub_id = get_azure_cli_credentials()
creds = credentials._get_cred(resource=None)
token = creds._token_retriever()[2]
access_token = token['accessToken']
header_token_value = "Bearer {}".format(access_token)

In [ ]:
name = "Bookmark test from notebook"
query = "AzureActivity | where TimeGenerated < ago(5d)"
entity_mappings = {}
entity_mappings.update({'550a6d02-d667-49d8-969a-e709cce03293': 'Account'})
entity_mappings.update({'201.12.34.111': 'Host'})
entities = r"{\"550a6d02-d667-49d8-969a-e709cce03293\": \"Account\", \"201.12.34.111\": \"Host\"}"
query_result = r"{\"Value\":0,\"Time\":\"2020-03-22T16:46:20.006499Z\",\"Legend\":\"F5Telemetry_LTM_CL\",\"__entityMapping\":" + entities + "}"
payload_data = "{\"properties\": { \"displayName\": \"" + name + "\", \"notes\": \"Testing from notebook\", \"labels\": [\"test\"], \"query\": \"" + query + "\", \"queryResult\": \"" + query_result + "\" }}"

In [ ]:
# Calling Microsoft Sentinel Watchlist API
response_bookmark = call_azure_rest_api(header_token_value, "bookmarks", payload_data, str(uuid.uuid4()), "2020-01-01")

In [ ]:
response_bookmark.text